In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

c:\users\acer\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\acer\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
df=pd.read_csv('eda_data.csv')

In [4]:
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,0,2015,1,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,0,2014,1,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,0,2013,1,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,0,2012,1,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,0,2011,1,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [5]:
df.shape

(2938, 22)

In [6]:
#Keeping the dataset of year 2014 and 2015 in test set

years_to_take = [2014, 2015]
df_test = df[df['Year'].isin(years_to_take)]


In [7]:
df_test.shape


(366, 22)

In [8]:
#Dropping entries from 2014 and 2015 from training set
df.drop(df[df['Year'] > 2013].index, inplace = True)

### Linear Regression

In [9]:
from sklearn.linear_model import Lasso,LinearRegression,Ridge
lr=LinearRegression().fit(df.drop(['Life expectancy'],axis=1),df['Life expectancy'])

In [10]:
lr.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy'])

0.8331125052215868

### Lasso Regression

In [11]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso=Lasso()
parameters={'alpha':[1e-08,1e-07,1e-06,1e-05,1e-04,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],'max_iter':[-1]}
lasso_regressor=GridSearchCV(lasso,parameters,cv=4)

lasso_regressor.fit(df.drop(['Life expectancy'],axis=1),df['Life expectancy'])
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

{'alpha': 0.0001, 'max_iter': -1}
0.8152799347036825


In [12]:
lasso=Lasso(alpha=0.0001,max_iter=-1)
lasso.fit(df.drop(['Life expectancy'],axis=1),df['Life expectancy'])

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=-1,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [13]:
lasso.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy'])

0.83310613660506

In [14]:
lasso.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy'])

0.83310613660506

### RandomForestRegression

In [15]:
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()
param_grid = { 
            "n_estimators" : [10,20,30,40,50,60,70,80,90,100],
            "criterion":['mse','gini'],
            'max_depth':[5,10,15,20,25,30],
            "max_features" : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8,10,12],
            "bootstrap": [True],
            }

grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=4)

grid.fit(df.drop(['Life expectancy'],axis=1),df['Life expectancy'])


GridSearchCV(cv=4, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, rand...e,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-1,
  

In [16]:
print(grid.best_params_)
print(grid.best_score_)

{'bootstrap': True, 'criterion': 'mse', 'max_depth': 25, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 50}
0.9023185946924106


In [17]:
regr = RandomForestRegressor(criterion='mse',max_depth=15,max_features='auto',n_estimators=80,bootstrap=True,min_samples_split=4)
regr.fit(df.drop(['Life expectancy'],axis=1),df['Life expectancy'])
regr.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy'])

0.9402811244815832

In [28]:
accuracies=pd.DataFrame([regr.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy']),lasso.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy']),lr.score(df_test.drop(['Life expectancy'],axis=1),df_test['Life expectancy'])],columns=['Accuracy'],index=['RandomForestRegressor','Lasso','LinearRegression'])
accuracies.head()

,Accuracy
RandomForestRegressor,0.940281
Lasso,0.833106
LinearRegression,0.833113


In [29]:
accuracies.to_csv('Accuracy.csv',index=False)

In [30]:
import pickle
with open('regr.pkl', 'wb') as file1:
    pickle.dump(regr, file1)